```
```

### backtesting 를 이용한 백테스팅 ###
---------------------------------------------------

In [1]:
!pip install backtesting

     ---------------------------------------- 0.0/175.5 kB ? eta -:--:--
     -------------------------------------- 175.5/175.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached bokeh-3.0.3-py3-none-any.whl (16.5 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached PyYAML-6.0-cp311-cp311-win_amd64.whl (143 kB)
     ---------------------------------------- 0.0/55.4 kB ? eta -:--:--
     ---------------------------------------- 55.4/55.4 kB ? eta 0:00:00
  Running setup.py install for backtesting: started
  Running setup.py install for backtesting: finished with status 'done'


  DEPRECATION: backtesting is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [1]:
import mariadb
import pandas   as pd
from datetime import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

c:\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
try:
    conn = mariadb.connect(
        user="root",
        password="123",
        host="localhost",
        port=3306,
        database="home"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")

cursor = conn.cursor()
cursor.execute("select * from daily_price where  CODE = ?",[('100840')])
data = cursor.fetchall()
desc = list(map(lambda x : x[0],cursor.description))
conn.close()

df =  pd.DataFrame(data=data, columns=desc)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='raise')
df = df.set_index(df['date'],drop=True)
df = df.drop(columns=['code','date','chg'])
df = df.rename(columns={'open':'Open','high':'High','low':'Low','close':'Close','volume':'Volume'})
df.head(5)

,Open,High,Low,Close,Volume
date,,,,,
2022-01-03,18850,18900,18600,18800,8954
2022-01-04,18850,18850,18600,18600,8138
2022-01-05,18750,18750,18450,18450,7794
2022-01-06,18200,18500,18000,18050,12756
2022-01-07,18050,18550,18050,18450,8534


pip install bokeh==2.4.3

In [25]:
import ta

def RSI(series, window=14):
    return ta.momentum.RSIIndicator(pd.Series(series), window, False).rsi()

class MyStrategy(Strategy) :
    
    def  init(self) :
        super().init()
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma60 = self.I(SMA, self.data.Close, 60)
        self.rsi = self.I(RSI, self.data.Close, 14)

    def next(self) :
        if self.rsi < 30 :
            self.buy()
        elif self.rsi > 70 :
            self.sell()

bt = Backtest(df, MyStrategy, cash=100_000_000,commission=.00015, exclusive_orders=True, trade_on_close=True)
stats = bt.run()
print(stats)
print(stats['_equity_curve'])
print(stats['_trades'])
bt.plot()

Start                     2022-01-03 00:00:00
End                       2023-02-17 00:00:00
Duration                    410 days 00:00:00
Exposure Time [%]                   70.967742
Equity Final [$]                 115788412.84
Equity Peak [$]                 140655813.005
Return [%]                          15.788413
Buy & Hold Return [%]                8.776596
Return (Ann.) [%]                   14.157372
Volatility (Ann.) [%]               43.563328
Sharpe Ratio                         0.324984
Sortino Ratio                        0.523139
Calmar Ratio                         0.490643
Max. Drawdown [%]                  -28.854755
Avg. Drawdown [%]                   -9.588094
Max. Drawdown Duration      184 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                   23
Win Rate [%]                        34.782609
Best Trade [%]                      24.763489
Worst Trade [%]                    -11.514053
Avg. Trade [%]                    

Row(id='8175', ...)